In [1]:
import numpy as np
from numpy.random import random_integers as rand
import pyquil
from pyquil.quil import Program
from pyquil.api import QVMConnection
from pyquil.gates import *
from scipy import linalg

qvm = QVMConnection()

In [39]:
def createToffoli():
    identity = np.identity(8)
    identity[[6],[6]] = 0
    identity[[6],[7]] = 1
    identity[[7],[7]] = 0
    identity[[7],[6]] = 1
    return identity

def inverseToffoli():
    identity = np.identity(8)
    identity[[0],[0]] = 0
    identity[[0],[1]] = 1
    identity[[1],[1]] = 0
    identity[[1],[0]] = 1
    return identity

def controlSWAPdef():
    identity = np.identity(8)
    identity[[5],[5]] = 0
    identity[[5],[6]] = 1
    identity[[6],[6]] = 0
    identity[[6],[5]] = 1
    return identity

def controlHadamard():
    identity = np.identity(4)
    identity[[2],[2]] = 1/np.sqrt(2)
    identity[[2],[3]] = 1/np.sqrt(2)
    identity[[3],[2]] = 1/np.sqrt(2)
    identity[[3],[3]] = -1/np.sqrt(2)
    return identity

def quantumRandomWalk(program):
    qvm = QVMConnection()
    program.inst(('Toffoli', 5,3,0))
 
    program.inst(('invToffoli', 5,3,0))
   
    program.inst(('Toffoli', 6,4,1))
 
    program.inst(('invToffoli', 6,4,1))

    program.inst(('Toffoli', 7,6,5))
 #   print(qvm.wavefunction(program))
    program.inst(('Toffoli', 8,7,2))
    print("After Toffoli",qvm.wavefunction(program))
    program.inst(('controlSWAP', 3,4,8))
    print("After CSWAP",qvm.wavefunction(program))
    program.inst(('Toffoli', 8,7,2))
    
    program.inst(('Toffoli', 7,6,5))
    
    program.inst(('invToffoli', 6,4,1))
    
    program.inst(('Toffoli', 6,4,1))
    
    program.inst(('invToffoli', 5,3,0))
    
    program.inst(('Toffoli', 5,3,0))
    print("After Toffoli", qvm.wavefunction(program))
    program.inst(('controlH', 4,3))
    print("End",qvm.wavefunction(program))
    return(program)

def quantumRandomWalk_nQubits(program,n):
        i=0
        Gx = i
        Gy = n+i
        A = 2*n
        Psi_x = 2*n+1+i
        Psi_y = 3*n+1+i
        ancilla1 = 4*n+1+i
        ancilla2 = 5*n+1+i
        ancilla3 = 6*n+1+i
        ancilla4  = 7*n+1+i
        
        program.inst(('Toffoli', Gx, Psi_x, ancilla1 ))
        
        program.inst(('invToffoli', Gx, Psi_x, ancilla1))
        
        program.inst(('Toffoli', Gy, Psi_y, ancilla2))
        
        program.inst(('invToffoli', Gy, Psi_y, ancilla2))
        
        program.inst(('Toffoli', ancilla1, ancilla2, ancilla3))
        
        program.inst(('Toffoli', A, ancilla3, ancilla4))

        program.inst(('controlSWAP', ancilla4, Psi_x, Psi_y))
        
        program.inst(('Toffoli', A, ancilla3, ancilla4))
        
        program.inst(('Toffoli', ancilla1, ancilla2, ancilla3))
        
        program.inst(('invToffoli', Gy, Psi_y, ancilla2))
        
        program.inst(('Toffoli', Gy, Psi_y, ancilla2))
        
        program.inst(('invToffoli', i, Psi_x, ancilla1))
        
        program.inst(('Toffoli', i, Psi_x, ancilla1))
        
        program.inst(('controlH', Psi_x, Psi_y))
        
        return(program)

def quantumRandomWalk_2QubitMaze(program):

    program.inst(('Toffoli', 0,3,5))
    program.inst(('invToffoli', 0,3,5))
    
    program.inst(('Toffoli', 1,4,6))
    program.inst(('invToffoli', 1,4,6))
    
    program.inst(('Toffoli', 5,6,7))
    program.inst(('Toffoli', 2,7,8))
    
    program.inst(('controlSWAP', 8,4,3))
    
    program.inst(('Toffoli', 2,7,8))
    program.inst(('Toffoli', 5,6,7))
    
    program.inst(('invToffoli', 1,4,6))
    program.inst(('Toffoli', 1,4,6))
    
    program.inst(('invToffoli', 0,3,5))
    program.inst(('Toffoli', 0,3,5))
    
    program.inst(('controlH', 3,4))
    
    return(program)

def defineGates(prog):
    controlH = controlHadamard()
    prog.defgate('controlH', controlH)

    controlSWAPgate = controlSWAPdef() 
    prog.defgate('controlSWAP', controlSWAPgate)

    invToffoli = inverseToffoli()
    prog.defgate('invToffoli', invToffoli)

    Toffoli =  createToffoli()
    prog.defgate('Toffoli', Toffoli)
    
    return(prog)


def initialise(prog):
    init_gate = np.identity(2**3)
    for i in [2,3,4,5]:
        init_gate[i,i] = 0.
    init_gate[2,3] = 1.
    init_gate[3,2] = 1.
    init_gate[4,5] = 1.
    init_gate[5,4] = 1.


    prog.defgate("INIT", init_gate)
    prog += H(0)
    prog += H(1) 
    prog.inst(("INIT",0,1,2))
    
    return prog
    #results = qvm.run(prog,[0,1,2],1)
    #print(results)

In [48]:
prog = Program()
prog = defineGates(prog)
prog = initialise(prog)

print(qvm.wavefunction(prog))
for i in range(3):
    #Start at room 1 -> Look to get to room 0
    prog += H(3)
    prog += H(4)
    prog = quantumRandomWalk_nQubits(prog,1)
prog.measure(3,3)
print(prog)

output = qvm.run(prog, [3], trials = 10000)
#print(output)
print(np.sum(output))

(0.5+0j)|000> + (0.5+0j)|011> + (0.5+0j)|101> + (0.5+0j)|110>
DEFGATE controlH:
    1.0, 0, 0, 0
    0, 1.0, 0, 0
    0, 0, 0.70710678118654746, 0.70710678118654746
    0, 0, 0.70710678118654746, -0.70710678118654746

DEFGATE controlSWAP:
    1.0, 0, 0, 0, 0, 0, 0, 0
    0, 1.0, 0, 0, 0, 0, 0, 0
    0, 0, 1.0, 0, 0, 0, 0, 0
    0, 0, 0, 1.0, 0, 0, 0, 0
    0, 0, 0, 0, 1.0, 0, 0, 0
    0, 0, 0, 0, 0, 0, 1.0, 0
    0, 0, 0, 0, 0, 1.0, 0, 0
    0, 0, 0, 0, 0, 0, 0, 1.0

DEFGATE invToffoli:
    0, 1.0, 0, 0, 0, 0, 0, 0
    1.0, 0, 0, 0, 0, 0, 0, 0
    0, 0, 1.0, 0, 0, 0, 0, 0
    0, 0, 0, 1.0, 0, 0, 0, 0
    0, 0, 0, 0, 1.0, 0, 0, 0
    0, 0, 0, 0, 0, 1.0, 0, 0
    0, 0, 0, 0, 0, 0, 1.0, 0
    0, 0, 0, 0, 0, 0, 0, 1.0

DEFGATE Toffoli:
    1.0, 0, 0, 0, 0, 0, 0, 0
    0, 1.0, 0, 0, 0, 0, 0, 0
    0, 0, 1.0, 0, 0, 0, 0, 0
    0, 0, 0, 1.0, 0, 0, 0, 0
    0, 0, 0, 0, 1.0, 0, 0, 0
    0, 0, 0, 0, 0, 1.0, 0, 0
    0, 0, 0, 0, 0, 0, 0, 1.0
    0, 0, 0, 0, 0, 0, 1.0, 0

DEFGATE INIT:
    1.0, 0,